The goal of this notebook is to test out morphologocal operations to isolate the quadrat. They're pretty finicky so I need to know how they work on all the test photos.

### Setup

In [1]:
from functions import reorder_file_paths
import cv2
from skimage import img_as_float, img_as_ubyte
from matplotlib import pyplot as plt
from functions import apply_otsu
from skimage.filters.rank import entropy
from skimage.morphology import disk, isotropic_closing, binary_closing
from scipy import ndimage
import numpy as np

In [2]:
# Determine the folder of images that will be looped through
path = 'test_photos/*'

# Get list of files for looping through
filenames = reorder_file_paths(path)

print(filenames)

['test_photos\\image_0.jpg', 'test_photos\\image_1.jpg', 'test_photos\\image_2.jpg', 'test_photos\\image_3.jpg', 'test_photos\\image_4.jpg', 'test_photos\\image_5.jpg', 'test_photos\\image_6.JPG', 'test_photos\\image_7.JPG', 'test_photos\\image_8.JPG', 'test_photos\\image_9.JPG', 'test_photos\\image_10.JPG', 'test_photos\\image_11.jpg', 'test_photos\\image_12.jpg', 'test_photos\\image_13.jpg', 'test_photos\\image_14.jpg', 'test_photos\\image_15.jpg', 'test_photos\\image_16.jpg', 'test_photos\\image_17.jpg', 'test_photos\\image_18.JPG', 'test_photos\\image_19.JPG', 'test_photos\\image_20.jpg', 'test_photos\\image_21.jpg', 'test_photos\\image_22.jpg', 'test_photos\\image_23.jpg', 'test_photos\\image_24.jpg']


### Use threshold to pick up on presence of quadrat

In [18]:
for i, filename in enumerate(filenames):
    # Load in the images in grayscale
    img = cv2.imread(filename, 0)
    
    # Convert images into floats
    img_float = img_as_float(img)

    # If image is <= to the threshold number, it's True. False otherwise.
    img_binary =  (img_float <= .9) # .90 is meant to pick up the white of the quadrat.

    # Convert back to ubyte
    img_ubyte = img_as_ubyte(img_binary)

    # Denoise the image to get rid of the grass that made it through
    img_median = ndimage.median_filter(img_ubyte, 9)
    # img_median_gau = ndimage.gaussian_filter(img_ubyte, 3)

    kernel = np.ones((3,3), np.uint8)

    # Dilate
    img_eroded = cv2.erode(img_median, kernel, iterations = 3)
    img_er_and_dilated = cv2.dilate(img_eroded, kernel, iterations = 1)

    # Use binary closing to potentially close gaps
    # mask = binary_closing(img_eroded, 2)

    # Apply orb corner detection
    # Initiate orb wowowowow
    orb = cv2.ORB_create(50)

    key_points, descriptors = orb.detectAndCompute(img_er_and_dilated, None)

    # Draw keypoints
    img_modified = cv2.drawKeypoints(img_er_and_dilated, key_points, None, flags=0)

    plt.imsave(f'results/isolating_quadrat/image_{i}_eroded_and_dilated.jpg', img_modified)

    # Display the images
#     cv2.imshow(f'img {i}', img_ubyte)

# cv2.waitKey(0)
# cv2.destroyAllWindows()